PARTIR PARA A PARTE DE RANKINGS

<h1 style='font-size:40px'>Process Automation Project </h1>
<ul style='font-size:20px'>
    <li>
        The goal of the present project is to explore Python's potential in solving automation tasks.
    </li>
    <li>
        We are going to deal with data from a clothing company. It has several stores across the Brazilian territory. 
    </li>
    <li>
        The brand's directory has asked us to produce a small daily and annual report on the sellings of the stores to be sent to their respective managers. Also, a ranking about their revenues must be created.
    </li>
</ul>

In [221]:
from pathlib import Path
import shutil
import os
import pandas as pd
import numpy as np
from email import encoders
import smtplib
import re
#os.chdir('Projeto AutomacaoIndicadores/')

<h2 style='font-size:30px'> <strong>Loading the files</strong></h2>

In [1]:
# Extracting the zip file containing the project's data.
from zipfile import ZipFile
with ZipFile('Projeto AutomacaoIndicadores.zip' , 'r') as zip_file:
    zip_file.extractall()

In [13]:
# Loading the csv file with the stores' id and shopping mall names where they are located.
import pandas as pd
stores = pd.read_csv('Bases de Dados/Lojas.csv', sep=';', encoding='latin1', index_col=0)
stores.head()

,Loja
ID Loja,
1,Iguatemi Esplanada
2,Shopping Midway Mall
3,Norte Shopping
4,Shopping Iguatemi Fortaleza
5,Shopping União de Osasco


In [355]:
# This xlsx file holds the stores' manager name and his/her email.
import re
stores_infos = pd.read_excel('Bases de Dados/Emails.xlsx', usecols=[0,1,2])
my_email='mypersonalemail@gmail.com'
# A small change: substituting the default email for one of my personal use.
for i, email in zip(stores_infos.index, stores_infos['E-mail']):
    stores_infos.loc[i, 'E-mail']= re.sub('pythonimpressionador', my_email, email)
    
stores_infos.head()

,Loja,Gerente,E-mail
0,Iguatemi Esplanada,Helena,mypersonalemail@gmail.com+helena@gmail.com
1,Shopping Midway Mall,Alice,mypersonalemail@gmail.com+alice@gmail.com
2,Norte Shopping,Laura,mypersonalemail@gmail.com+laura@gmail.com
3,Shopping Iguatemi Fortaleza,Manuela,mypersonalemail@gmail.com+manuela@gmail.com
4,Shopping União de Osasco,Valentina,mypersonalemail@gmail.com+valentina@gmail.com


In [6]:
# This last file stores the selling records from all the stores
sellings = pd.read_excel('Bases de Dados/Vendas.xlsx')
sellings.head()

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final
0,1,2019-01-01,1,Sapato Estampa,1,358,358
1,1,2019-01-01,1,Camiseta,2,180,360
2,1,2019-01-01,1,Sapato Xadrez,1,368,368
3,2,2019-01-02,3,Relógio,3,200,600
4,2,2019-01-02,3,Chinelo Liso,1,71,71


<h2 style='font-size:30px'> Backup Data</h2>
<div>
    <ul style='font-size:20px'>
        <li>
            Before doing the main tasks demanded, let's firstly generate a backup directory where we'll place all the selling accounts from each of the shops.
        </li>
    </ul>
</div>

In [ ]:
! mkdir Backup\ Arquivos\ Lojas

In [30]:
# Creating a backup directory for the records from every single company's stores.
from pathlib import Path
import os
for store in stores.Loja:
    os.mkdir(f'Backup Arquivos Lojas/{store}')

In [236]:
# To facilitate the execution of our tasks, I've decided to make this dictionary containing the stores' id 
# and their respective names {store_id : store_name}.
stores_dict = stores.to_dict()['Loja']
stores_dict

{1: 'Iguatemi Esplanada',
 2: 'Shopping Midway Mall',
 3: 'Norte Shopping',
 4: 'Shopping Iguatemi Fortaleza',
 5: 'Shopping União de Osasco',
 6: 'Shopping Center Interlagos',
 7: 'Rio Mar Recife',
 8: 'Salvador Shopping',
 9: 'Rio Mar Shopping Fortaleza',
 10: 'Shopping Center Leste Aricanduva',
 11: 'Ribeirão Shopping',
 12: 'Shopping Morumbi',
 13: 'Parque Dom Pedro Shopping',
 14: 'Bourbon Shopping SP',
 15: 'Palladium Shopping Curitiba',
 16: 'Passei das Águas Shopping',
 17: 'Center Shopping Uberlândia',
 18: 'Shopping Recife',
 19: 'Shopping Vila Velha',
 20: 'Shopping SP Market',
 21: 'Shopping Eldorado',
 22: 'Shopping Ibirapuera',
 23: 'Novo Shopping Ribeirão Preto',
 24: 'Iguatemi Campinas',
 25: 'Shopping Barra'}

In [52]:
# And now, we are going to split up the data from 'sellings'. A csv file will be generated holding
# each of the stores' selling records by day.

# Note: in Portuguese, 'Data' has the same meaning as the word 'Date'.
for df in sellings.groupby(['ID Loja', 'Data']):
    store_id = df[0][0]
    date = df[0][1].date()
    # The dicionary just created will be used in order to place the stores' names in the csv file's name using their id.
    df[1].to_csv(f'Backup Arquivos Lojas/{stores_dict[store_id]}/{date}-{stores_dict[store_id]}.csv')
    
# With this ended, we are able to move to the main tasks execution.

<center>
    <img src='daily_report.png'>
</center>
<ul style='font-size:20px'>
    <li>
        Now all the shops will have a daily account on all their sellings made.
    </li>
</ul>

<h1>Creating the vendors ranking</h1>
<div>
    <ul style='font-size:20px'>
        <li>
            We need to provide to the company's leaders a daily and an annual ranking concerning the vendors' revenue
        </li>
    </ul>
</div>

In [ ]:
# Before diving into the solution of the task, I've found adequate to create a directory in which the ranking will be placed.
! mkdir Rankings

In [ ]:
# The daily rankings can be readily created by performing a groupby in the 'sellings' DF.
daily_rankings = sellings.groupby(['Data', 'ID Loja'])['Valor Final'].sum()

# Renaming the stores' ID column for the sake of better comprehension of the document to be sent.
daily_rankings.rename(index = stores_dict, inplace = True)

# For every date in the sellings data, we are going to generate a revenue ranking among the vendors. They will be stored in the
# just created 'Rankings'directory.
for date in sellings.Data:
    daily_rankings.xs(date, level=0).sort_values(ascending=False).to_csv(f'Rankings/{date.date()}-Ranking.csv')

In [311]:
# Now, we'll be able to send the daily rankings to the company's leadership.
# Importing the libraries needed for the email sendings.
from email.message import EmailMessage
import smtplib
import mimetypes
rankings_dir = Path('Rankings')
directory_email = stores_infos.iloc[-1,2]

# We'll iterate over the 'Rankings' directory and grab each file that is there.
for ranking in rankings_dir.iterdir():
    
    # Picking the file's date and creating the email message.
    date = re.search('2019-\d{2}-\d{2}', ranking.name)[0]
    msg = EmailMessage()
    msg['From'] = os.environ.get('EMAIL_USER')
    msg['To'] = directory_email
    msg['Subject'] = f'{date} performance ranking'
    msg.add_alternative(f''' 
    <p>Dear directory,</p>
    <p>As solicited, here is the stores performance ranking from {date}</p>
    
    <p>Best regards, </p>
    <p>The Data Analysis Team, </p>
    
    ''', subtype='html')
    
    # Attaching the ranking csv and finally sending the message.
    with open(ranking, 'rb') as file:
        file_content = file.read()
        maintype, subtype = mimetypes.guess_type(ranking)[0].split('/')
        msg.add_attachment(file_content, maintype = maintype, subtype=subtype, filename=file.name)
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(os.environ.get('EMAIL_USER'), os.environ.get('MOT_USER'))
            smtp.send_message(msg)
    

SMTPResponseException: (250, b'2.1.0 OK q6-20020a056870028600b000d9be0ee766sm3264284oaf.57 - gsmtp')

<ul style='font-size:20px'>
    <li>
        The email received is intended to look like as the one below.
    </li>
</ul>
<center>
    <img src='daily_ranking.png'>
</center>

In [319]:
# Making the annual ranking. Again, a simple groupby will satisfactorily handle the job for us.
annual_ranking = sellings.groupby('ID Loja')['Valor Final'].sum()
annual_ranking.rename(index=stores_dict, inplace=True)
annual_ranking.to_csv('Rankings/2019-Overall-Ranking.csv')

In [325]:
# Now, we'll make an almost identical process we've performed concerning the daily rankings.
directory_email = stores_infos.iloc[-1,2] 
rankings_dir_files = os.listdir(Path('Rankings'))

# Searching among the 'Rankings' directory files the Annual Ranking csv.
for file in rankings_dir_files:
    if '2019-Overall-Ranking.csv' in file:
        
        # Once the document has been found, it will be attached to the email message.
        ranking = 'Rankings/2019-Overall-Ranking.csv'
        msg = EmailMessage()
        msg['From'] = os.environ.get('EMAIL_USER')
        msg['To'] = directory_email
        msg['Subject'] = '2019 performance ranking'
        msg.add_alternative(
        '''
        <p>Dear directory,</p>
        <p>The annual stores performance report has been finished, we'll be sending it to you as an
        attachment.</p>
        <p>Best regards,</p>
        <p>The data Analysis Team </p>
        
        ''', subtype='html')
        with open(ranking, 'rb') as file:
            file_content = file.read()
            maintype, subtype = mimetypes.guess_type(ranking)[0].split('/')
            msg.add_attachment(file_content, maintype = maintype, subtype=subtype, filename=file.name)
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(os.environ.get('EMAIL_USER'), os.environ.get('MOT_USER'))
            smtp.send_message(msg)

<ul style='font-size:20px'>
    <li>
        The email received is intended to look like as the one below.
    </li>
</ul>
<center>
    <img src='annual_ranking.png'>
</center>

<h1> Generating the daily and yearly One Page reports</h1>
<ul style='font-size:20px'>
    <li>
        For every day in 2019, we ought to engender a report for the vendors displaying how well or bad they were in achieving the company's sales target metrics.
    </li>
    <li>
        The metrics were: earn a revenue above 1000.00 reais; sell at least four different kinds of products; and get an average selling ticket of at least 500.00 reais.
    </li>
    <li>
        When it comes to the overall performance during the year, the vendors must have earned a revenue above 1,650,00.00 reais; sold at least 120 different kinds of products; and got an average selling ticket of at least 500.00 reais.
    </li>
</ul>

In [351]:
# Iterating over the stores' names contained in the 'stores_dict' dictionary.
for store in stores_dict.values():
    
    # Getting the manager's name and email.
    manager = stores_infos[stores_infos['Loja'] == store]['Gerente'].values[0]
    manager_email =  stores_infos[stores_infos['Loja'] == store]['E-mail']
    
    # Iterating over the shop's directory in order to extract the data from the daily reports we've generated.
    store_directory = Path(f'Backup Arquivos Lojas/{store}')
    for csv in store_directory.iterdir():
        date = re.search('2019-\d{2}-\d{2}', str(csv.name))[0]
        
        # Opening the report and calculating the metrics needed for the One Page report.
        report_day = pd.read_csv(csv)
        revenue = report_day['Valor Final'].sum()
        products = len(report_day['Produto'].unique())
        revenue_per_selling = report_day.groupby(['Código Venda'])['Valor Final'].sum()
        average_ticket = revenue_per_selling.mean()
        
        # With the parameters' values obtained, it is time to write the email text and send it to the manager's address.
        msg = EmailMessage()
        msg['From'] = os.environ.get('EMAIL_USER')
        msg['To'] = manager_email
        msg['Subject'] = f'{date} One Page Report'
        msg.add_alternative(
                            f'''
            <html>
                    <p> Dear {manager},</p>
                    <p>Here is your store's sellings performance report on the day {date}. </p>
                    <table>
                        <tr>
                            <th>Metric</th>
                            <th>Target Value</th>
                            <th>Value Obtained</th>
                            <th>Metric Status</th>
                        </tr>
                        <tr>
                            <td>Revenue</td>
                            <td>R$ 1,000.00</td>
                            <td>R$ {revenue:,.2f} </td>
                            <td> <font color='{'green' if revenue >= 1000 else 'red'}'>{'Achieved' if revenue >= 1000 else 'Failed'} </font></td>
                        </tr>

                        <tr>
                            <td>No of Diff. Products</td>
                            <td>4</td>
                            <td>{products} </td>
                            <td> <font color='{'green' if products >= 4 else 'red'}'> {'Achieved' if products >= 4 else 'Failed'}</font></td>
                        </tr>

                        <tr>
                            <td>Average Ticket</td>
                            <td>R$ 500.00</td>
                            <td>R$ {average_ticket:,.2f} </td>
                            <td> <font color='{'green' if average_ticket >= 500 else 'red'}'>{'Achieved' if average_ticket >= 500 else 'Failed'} </font></td>

                        </tr>
                    </table>
                    <p> Best regards,</p>
                    <p> The Data Analysis Team</p>
            </html>
                    ''', subtype='html')
        with open(csv, 'rb') as csv_file:
            file_content = csv_file.read()
            maintype, subtype = mimetypes.guess_type(csv)[0].split('/')
            msg.add_attachment(file_content, maintype = maintype, subtype=subtype, filename=csv.name)


        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(os.environ.get('EMAIL_USER'), os.environ.get('MOT_USER'))
            smtp.send_message(msg)

SMTPResponseException: (250, b'2.0.0 OK  1649200013 r205-20020acadad6000000b002ef824213c9sm5911984oig.55 - gsmtp')

<ul style='font-size:20px'>
    <li>
        The emails received should be similar to the one below.
    </li>
</ul>
<center>
    <img src='onepage1.png'>
</center>

In [353]:
# A similar process will be done when creating the Annual One Page Report.
annual_revenues = sellings.groupby(['ID Loja'])['Valor Final'].sum()
products =  sellings.groupby(['ID Loja'])['Produto'].nunique()
revenue_per_selling = sellings.groupby(['ID Loja' ,'Código Venda'])['Valor Final'].sum()

for id_store, store in stores_dict.items():
    # Getting the manager's name and email and the metrics wished.
    manager = stores_infos[stores_infos['Loja'] == store]['Gerente'].values[0]
    manager_email =  stores_infos[stores_infos['Loja'] == store]['E-mail']
    annual_revenue = annual_revenues.loc[id_store]
    diff_products = products.loc[id_store]
    average_ticket = revenue_per_selling.xs(id_store, level=0).mean()
    
    # Now, we'll write the email to be sent to the managers.
    msg = EmailMessage()
    msg['From'] = os.environ.get('EMAIL_USER')
    msg['To'] = manager_email
    msg['Subject'] = f'2019 One Page Report'
    msg.add_alternative(
                        f'''
        <html>
                <p> Dear {manager},</p>
                <p>Here is your store's annual sellings performance report. </p>
                <table>
                    <tr>
                        <th>Metric</th>
                        <th>Target Value</th>
                        <th>Value Obtained</th>
                        <th>Metric Status</th>
                    </tr>
                    <tr>
                        <td>Revenue</td>
                        <td>R$ 1,650,000.00</td>
                        <td>R$ {annual_revenue:,.2f} </td>
                        <td> <font color='{'green' if annual_revenue >=  1650000 else 'red'}'>{'Achieved' if annual_revenue >= 1650000 else 'Failed'} </font></td>
                    </tr>

                    <tr>
                        <td>No of Diff. Products</td>
                        <td>120</td>
                        <td>{diff_products} </td>
                        <td> <font color='{'green' if diff_products >= 120 else 'red'}'> {'Achieved' if diff_products >= 120 else 'Failed'}</font></td>
                    </tr>

                    <tr>
                        <td>Average Ticket</td>
                        <td>R$ 500.00</td>
                        <td>R$ {average_ticket:,.2f} </td>
                        <td> <font color='{'green' if average_ticket >= 500 else 'red'}'>{'Achieved' if average_ticket >= 500 else 'Failed'} </font></td>

                    </tr>
                </table>
                <p> Best regards,</p>
                <p> The Data Analysis Team</p>
        </html>
                ''', subtype='html')
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(os.environ.get('EMAIL_USER'), os.environ.get('MOT_USER'))
            smtp.send_message(msg)

SMTPResponseException: (250, b'smtp.gmail.com at your service, [2804:431:c7f4:ff25:4143:f612:5e5d:1842]\nSIZE 35882577\n8BITMIME\nAUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8')

<ul style='font-size:20px'>
    <li>
        The emails received should be similar to the one below.
    </li>
</ul>
<center>
    <img src='annual_onepage.png'>
</center>